**Updated on:** 2022-10-23 23:10:05 CEST

This Notebook is used for cleaning the feature table, an output of metabolomics experiment, then performing some preliminary univariate and multivariate statistics analyses.

**Authors**: Abzer Kelminal (abzer.shah@uni-tuebingen.de), Francesco Russo (frru@ssi.dk), Filip Ottosson (faot@ssi.dk), Madaleine Ernst (maet@ssi.dk), Axel Walter (axel.walter@uni-tuebingen.de), Carolina Gonzalez, Judith Boldt <br>
**Input file format**: .csv files or .txt files <br>
**Outputs**: .csv files, .pdf & .svg images  <br>
**Dependencies**: pandas numpy plotly pingouin kaleido scikit-learn

---
This Notebook can be run with both Jupyter Notebook & Google Colab. To know more about how to get the Jupyter Notebook running with R code, please have a look at this document: [GitHub Link](https://github.com/Functional-Metabolomics-Lab/Jupyter-Notebook-Installation/blob/main/Anaconda%20with%20R%20kernel%20installation.pdf)

---
**Before starting to run this notebook with your own data, remember to save a copy of this notebook in your own Google Drive! Do so by clicking on File --> Save a copy in Drive. You can give whatever meaningful name to your notebook.** This file should be located in a new folder of your Google Drive named 'Colab Notebooks'. You can also download this notebook: File --> Download --> Download .ipynb.<br>

---
<b><font size=3> SPECIAL NOTE: Please read the comments before proceeding with the code and let us know if you run into any errors and if you think it could be commented better. We would highly appreciate your suggestions and comments!!</font> </b>

---

# **About the Data**

The files used in this tutorial are part of an interlab comparison study, where different laboratories around the world analysed the same environmental samples on their respective LC-MS/MS equipments. To simulate algal bloom, standardized algae extracts (A) in marine dissovled organic matter (M) at different concentrations were prepared (450 (A45M); 150 (A15M); and 50 (A5M) ppm A). Samples were then shipped to different laboratories for untargeted LC-MS/MS metabolomics analysis. The data used particularly for this notebook is from Lab 1 (Dorrestein Lab, University of California at San Diego, USA; Data submitted by Allegra Aron allegra.aron@gmail.com ) <br><br>
(*To be edited*) In this tutorial, we are working with one of the datasets, which was acquired on a UHPLC system coupled to a Thermo Scientific Q Exactive HF Orbitrap LC-MS/MS mass spectrometer. MS/MS data were acquired in data-dependent acquisition (DDA) with fragmentation of the five most abundant ions in the spectrum per precursor scan. Data files were subsequently preprocessed using [MZmine3](http://mzmine.github.io/) and the [feature-based molecular networking workflow in GNPS](https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=d207c3a831264d61810ad69ac09b14e9).

# **About the different sections in the Notebook:**
### **1. Data-cleaning**

It involves cleaning the feature table, which contains all the features (metabolites, in our case) with their corresponding intensities. The data cleanup steps involved are: 1) Blank removal 2) Imputation 3) Normalisation. Each step would be discussed in detail later. Once the data is cleaned, we can then use it for further statistical analyses.

### **2. Univariate statistical analysis**

Here, we will use univariate statistical methods, such as ANOVA, to investigate whether there are differences in the levels of individual features between different time points in the dataset.

### **3. Unsupervised multivariate analyses:**
#### **i. PCoA and PERMANOVA**
Here, we will perform a Principal Coordinate Analysis (PCoA), also known as metric or classical Multidimensional Scaling (metric MDS) to explore and visualize patterns in an untargeted mass spectromtery-based metabolomics dataset. We will then assess statistical significance of the patterns and dispersion of different sample types using permutational multivariate analysis of variance (PERMANOVA).

#### **ii. Cluster Analyses and Heatmaps**
We will also perform different cluster analyses to explore patterns in the data. This will help us to discover subgroups of samples or features that share a certain level of similarity. Clustering is an example of unsupervised learning where no labels are given to the learning algorithm which will try to find patterns/structures in the input data on its own. The goal of clustering is to find these hidden patterns.<br>

Some types of cluster analyses (e.g. hierarchical clustering) are often associated with heatmaps. Heatmaps are a visual representation of the data where columns are usually samples and rows are features (in our case, different metabolic features). The color scale of heatmaps indicates higher or lower intensity (for instance, blue is lower and red is higher intensity).<br>

There are a lot of good videos and resources out there explaining very well the principle behind clustering. Some good ones are the following:<br>
- Hierarchical clustering and heatmaps: https://www.youtube.com/watch?v=7xHsRkOdVwo<br>
- K-means clustering: https://www.youtube.com/watch?v=4b5d3muPQmA

# **Questions to be asked in the Statistical analysis sections**: </br>
**Univariate Statistical analysis:**
*   Are metabolite levels dependent on the dilution?
*   How does the affected metabolite change throughout the dilution series?
*   How large are the differences? 
---
**Unsupervised multivariate analyses: PCoA & PERMANOVA**
*   Can we monitor algal bloom by looking at metabolomic profiles of marine dissolved organic matter?
---
**Cluster analysis and Heatmaps**
- Can we monitor algal bloom by looking at metabolomic profiles of marine dissolved organic matter?
- Are we able to group/cluster together samples derived from different concentrations of algae extracts using metabolic profiles? <br>
- Which samples are the most similar? <br>
- Are there any patterns defining the groups/clusters? That is, which features cluster together? 

# **Package installation:**
Since we are running the notebook via Colab environment which runs completely in cloud, we need to install the packages every time we run the notebook.This might take some time to install all these packages. In case you are running the notebook directly via Jupyter Notebook IDE, you need to install the packages only once.

In [ ]:
# Install libraries that are not preinstalled
!pip install pandas numpy plotly scikit-learn scikit-bio pingouin kaleido ipyfilechooser nbformat

In [ ]:
# importing necessary modules
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
from sklearn.decomposition import PCA
import pingouin as pg
import skbio # Don't import on Windows!!
from ipyfilechooser import FileChooser
from ipywidgets import interact
import warnings

In [ ]:
# Disable warnings for cleaner output, comment out for debugging
warnings.filterwarnings('ignore')

# **Setting a local working directory:**
### For Google Colab Users:
<p style='text-align: justify;'> <font color='red'>For Google Colab, it is not possible to access the files from your local computer as it is hosted on Google's cloud server. An easier workaround is to upload the necessary files into the Google colab session using the 'Files' icon on the left as shown in the image. The code in the next cell creates a new folder 'My_TestData' in the Colab space and sets the folder as working directory. Following the steps in the image, you can check in your Colab to see if the folder has been created. Once you see it, simply upload the files from your local PC to the folder 'My_TestData' and then continue running the rest of the script.</font> </p>

<p style='text-align: justify;'><b>SPECIAL NOTE: All the files uploaded to Google Colab would generally disappear after 12 hours. Similarly, all the outputs would be saved only in the Colab, so we need to download them into our local system at the end of our session.</b></p> 

[Go to section: Getting outputs from Colab](#colab_output) 

**Importing files into Google Colab environment:**
![Google-Colab Files Upload](https://github.com/abzer005/Images-for-Jupyter-Notebooks/blob/main/StepsAll.png?raw=true)

In [ ]:
# Get folder with data files
result_dir = input("Enter path to folder for your results (or leave empty to stay in this folder):\n")
if not result_dir:
    result_dir = "."
if not os.path.exists(result_dir):
    os.mkdir(result_dir)
print(f"Results folder is: {os.path.abspath(result_dir)}")

**For users running the script directly in Jupyter Notebook instead through Google Colab**, please make sure to include all the input files in one folder before running the script. Then for setting the working directory, use the below code on a new cell. When you run the cell, it will display an output box where you can enter the path of the folder containing all your input files in your local computer and it will set as your working directory<br> For ex: D:\User\Project\Test_Data

```
directory = input("Enter the path of the folder with input files:\n")
os.chdir(directory)
```



# **Input files needed for the Notebook:**
1) <b>Feature table:</b> An output of metabolomics experiment, containing all the features or peaks (LC-MS/MS peaks here) with their corresponding intensities. The feature table used in the test data is obtained by MZmine3. (Filetype: .csv file) </br> 
2) <b>Metadata:</b> Created by the user about the files used obtaining the feature table (It can be a csv/txt/tsv file). The columns in a metadata should be created with the following format: filename (1st column having all the filenames in the same order as the columns in feature table), all the other columns with column name such as: ATTRIBUTE_yourDesiredAttribute. </br>

Please have a look at the metadata used here for reference. Creating a metadata in the above-mentioned format is necessary for uploading the files in GNPS and to obtain a molecular network.

## Reading the input data using URL:
Here, we can directly pull **example data** files from our Functional Metabolomics GitHub page.

In [ ]:
#Reading the input data using URL 
ft_url = 'https://raw.githubusercontent.com/Functional-Metabolomics-Lab/Statistical-analysis-of-non-targeted-LC-MSMS-data/main/data/SD_BeachSurvey_GapFilled_quant.csv'
md_url = 'https://raw.githubusercontent.com/Functional-Metabolomics-Lab/Statistical-analysis-of-non-targeted-LC-MSMS-data/main/data/20221125_Metadata_SD_Beaches_with_injection_order.txt'

ft = pd.read_csv(ft_url)
md = pd.read_csv(md_url, sep = "\t").set_index("filename")

Specify your own feauture quantification and meta data table. If not, the example data will be used.

In [ ]:
# feature quantification table file location
ft_file = ""
# meta data table file location
md_file = ""


# define separators for different input file formats
separators = {"csv": ",", "tsv": "\t", "txt": "\t"}

# read feature table
if ft_file:
    ft = pd.read_csv(ft_file, sep = separators[file.split(".")[-1]])
else:
    print("Please select a feature file and rerun this cell.")
# read metadata table
if md_file:
    md = pd.read_csv(md_file, sep = separators[file.split(".")[-1]]).set_index("filename")
else:
    print("Please select a metavalue file and rerun this cell.")

Let's check if the data has been read correclty!!

In [ ]:
print('Dimension: ',ft.shape) #gets the dimension (number of rows and columns) of ft
ft.head() # gets the first 5 rows of ft

In [ ]:
print('Dimension: ',md.shape)
md.head()

## **Creating Functions:**
<p style='text-align: justify;'> Before getting into the Data cleanup steps, we have created a function that can be used later for data summarization. By creating functions, we don't have to write these big codes multiple times. Instead, we just use the function name. <font color="red">The following cell in this section will not produce any outputs here. </font> The outputs will be produced when we give input variables to the function in the later sections. </p>

<p style='text-align: justify;'> Using this function InsideLevels, we get an idea of the multiple levels in each of the metioned attributes in the metadata as well as the datatype of each attribute.  <font color ="blue"> This function takes metadata table as its input. </font></p>

In [ ]:
def inside_levels(df):
    # get all the columns (equals all attributes) -> will be number of rows
    levels = []
    types = []
    count = []
    for col in df.columns:
        types.append(type(df[col][0]))
        levels.append(sorted(set(df[col].dropna())))
        tmp = df[col].value_counts()
        count.append([tmp[levels[-1][i]] for i in range(len(levels[-1]))])
    return pd.DataFrame({"ATTRIBUTES": df.columns, "LEVELS": levels, "COUNT":count, "TYPES": types}, index=range(1, len(levels)+1))

First, let's have a look at the different conditions within each attribute of our metadata.

In [ ]:
inside_levels(md)

The above table is a summary of our metadata tabel. For example, the 1st row says that there are 5 different types of sample under 'ATTRIBUTE_Sample' category namely A15M,A45M,A5M,M,PPL and the count of each of these types is 3,3,3,1.

# **Arranging metadata and feature table in the same order:**

<p style='text-align: justify;'> In the next cell, we are trying to bring the feature table and metadata in the correct format such as <font color ="green"> the rownames of metadata and column names of feature table are the same. </font> They both are the file names and they need to be the same, as from now on, we will call the columns in our feature table based on our metadata information. Thus, using the metadata, the user can filter their data easily. You can also directly deal with your feature table without metadata by getting your hands dirty with some coding!! But having a metadata improves the user-experience greatly. </p>

In [ ]:
# structure of the original metadata file
md.head()

In [ ]:
new_md = md.copy() #storing the files under different names to preserve the original files
# remove the (front & tail) spaces, if any present, from the rownames of md
new_md.index = [name.strip() for name in md.index]
# for each col in new_md
# 1) removing the spaces (if any)
# 2) replace the spaces (in the middle) to underscore
# 3) converting them all to UPPERCASE
for col in new_md.columns:
    if new_md[col].dtype == str:
        new_md[col] = [item.strip().replace(" ", "_").upper() for item in new_md[col]]
print('Dimension: ',new_md.shape)
new_md.head()

In [ ]:
# structure of the original feature file
ft.head()

In [ ]:
new_ft = ft.copy() #storing the files under different names to preserve the original files
# changing the index in feature table to contain m/z and RT information
new_ft.index = [f"{id}_{round(mz, 3)}_{round(rt, 3)}" for id, mz, rt in zip(ft["row ID"], ft["row m/z"], ft["row retention time"])]
# drop all columns that are not mzML or mzXML file names
new_ft.drop(columns=[col for col in new_ft.columns if ".mz" not in col], inplace=True)
# remove " Peak area" from column names
new_ft.rename(columns={col: col.replace(" Peak area", "").strip() for col in new_ft.columns}, inplace=True)
print('Dimension: ',new_ft.shape)
new_ft.head()

Checking the tables:

In [ ]:
# check if new_ft column names and md row names are the same
if sorted(new_ft.columns) == sorted(new_md.index):
    print(f"All {len(new_ft.columns)} files are present in both new_md & new_ft.")
else:
    print("Not all files are present in both new_md & new_ft.\n")
    # print the md rows / ft column which are not in ft columns / md rows and remove them
    ft_cols_not_in_md = [col for col in new_ft.columns if col not in new_md.index]
    print(f"These {len(ft_cols_not_in_md)} columns of feature table are not present in metadata table and will be removed:\n{', '.join(ft_cols_not_in_md)}\n")
    new_ft.drop(columns=ft_cols_not_in_md, inplace=True)
    md_rows_not_in_ft = [row for row in new_md.index if row not in new_ft.columns]
    print(f"These {len(md_rows_not_in_ft)} rows of metadata table are not present in feature table and will be removed:\n{', '.join(md_rows_not_in_ft)}\n")
    new_md.drop(md_rows_not_in_ft, inplace=True)

In [ ]:
new_ft = new_ft.reindex(sorted(new_ft.columns), axis=1) #ordering the ft by its column names
new_md.sort_index(inplace=True) #ordering the md by its row names

In [ ]:
# checking the dimensions of our new ft and md
print(f"The number of rows and columns in our original ft is: {ft.shape}")
print(f"The number of rows and columns in our new ft is: {new_ft.shape}")
print(f"The number of rows and columns in our original md is: {md.shape}")
print(f"The number of rows and columns in our new md is: {new_md.shape}\n")

Notice that the number of columns of feature table is same as the number of rows in our metadata. Now, we have both our feature table and metadata in the same order.

In [ ]:
#checking if they the files are in the same order
list(new_ft.columns) == list(new_md.index)

Lets check the files once again!!

In [ ]:
print('Dimension: ',new_ft.shape)
new_ft.head()

In [ ]:
print('Dimension: ',new_md.shape)
new_md.head()

# Splitting the data into Blanks and Samples using Metadata:
<a id="data_split"></a>

For the first step: Blank removal, we need to split the data as spectra obtained from blanks and samples respectively using the metadata. More about Blank removal in the next section.

In [ ]:
inside_levels(new_md)

In case we want to remove certain files of a particular condition, for ex: ATTRIBUTE_sample = "M", we can subset them out of our dataframe using the next cell. 

In [ ]:
# subset_data = new_md[new_md['ATTRIBUTE_Sample']!='M']
# print('Dimension: ',subset_data.shape)
# inside_levels(subset_data)

Once we subset the data, we can further proceed to split the blanks from the sample in the cell below. If no subsetting is involved, you can simply split your metadata into blank and sample.

In [ ]:
#If subset_data exists, it will take it as "data", else take new_md as "data"
if 'subset_data' in locals():
    data = subset_data
else:
    data = new_md
display(inside_levels(data))

condition = int(input("Enter the index number of the attribute to split sample and blank: "))
df = pd.DataFrame({"LEVELS": inside_levels(data).iloc[condition-1]["LEVELS"]})
df.index = [*range(1, len(df)+1)]
display(df)

#Among the shown levels of an attribute, select the ones to keep
blank_id = int(input("Enter the index number of your BLANK: "))
print('Your chosen blank is: ', df['LEVELS'][blank_id])

#Splitting the data into blanks and samples based on the metadata
md_blank = data[data[inside_levels(data)['ATTRIBUTES'][condition]] == df['LEVELS'][blank_id]]
blank = new_ft[list(md_blank.index)]
md_samples = data[data[inside_levels(data)['ATTRIBUTES'][condition]] != df['LEVELS'][blank_id]]
samples = new_ft[list(md_samples.index)]

In [ ]:
# Display the chosen blank
print('Dimension: ',blank.shape)
blank.head()

In [ ]:
# Display the chosen samples
print('Dimension: ',samples.shape)
samples.head()

**Now that we have our data ready, we can start with the cleanup steps!!**

# Step1: Blank Removal

<p style='text-align: justify;'> In LC-MS/MS, we use solvents called Blanks which are usually injected time-to-time to prevent carryover of the sample. The features coming from these Blanks would also be detected by LC-MS/MS instrument. Our goal here is to remove these features from our samples. The other blanks that can be removed are: Signals coming from growth media alone in terms of microbial growth experiment, signals from the solvent used for extraction methods and so on. Therefore, it is best practice to measure mass spectra of these blanks as well in addition to your sample spectra. </p>

**How do we remove these blank features?** </br> 
<p style='text-align: justify;'> Since we have the feature table split into Control blanks and Sample groups now, we can compare blanks to the sample to identify the background features coming from blanks. A common filtering method is to use a cutoff to remove features that are not present sufficient enough in our biological samples. </p>

The steps followed in the next few cells are:
1. <p style='text-align: justify;'> We find an average for all the feature intensities in your blank set and sample set. Therefore, for n no.of features in a blank or sample set, we get n no.of averaged features. </p>
2. <p style='text-align: justify;'> Next, we get a ratio of this average_blanks vs average_sample. This ratio Blank/sample tells us how much of that particular feature of a sample gets its contribution from blanks. If it is more than 30% (or Cutoff as 0.3), we consider the feature as noise. </p>
3. <p style='text-align: justify;'> The resultant information (if ratio > Cutoff or not) is stored in a bin such as 1 = Noise or background signal, 0 = Feature Signal</p>
4. <p style='text-align: justify;'> We count the no.of features in the bin that satisfies the condition ratio > cutoff, and consider those features as 'noise or background features' and remove them. </p>

**<font color='red'> The Cutoff used to obtain the all the files in MZmine Results folder is 0.3 </font>**

In [ ]:
blank_removal = samples.copy()
if (input("Do you want to perform Blank Removal- Y/N: ").upper()=="Y"):
    
    # When cutoff is low, more noise (or background) detected; With higher cutoff, less background detected, thus more features observed
    cutoff = float(input("Enter Cutoff value between 0.1 & 1 (Ideal cutoff range: 0.1-0.3): ")) # (i.e. 10% - 100%). Ideal cutoff range: 0.1-0.3
    
    # Getting mean for every feature in blank and Samples
    avg_blank = blank.mean(axis=1, skipna=False) # set skipna = False do not exclude NA/null values when computing the result.
    avg_samples = samples.mean(axis=1, skipna=False)

    # Getting the ratio of blank vs samples
    ratio_blank_samples = (avg_blank+1)/(avg_samples+1)

    # Create an array with boolean values: True (is a real feature, ratio<cutoff) / False (is a blank, background, noise feature, ratio>cutoff)
    is_real_feature = (ratio_blank_samples<cutoff)

    # Checking if there are any NA values present. Having NA values in the 4 variables will affect the final dataset to be created
    temp_NA_Count = pd.concat([avg_blank, avg_samples, ratio_blank_samples, is_real_feature], 
                            keys=['avg_blank', 'avg_samples', 'ratio_blank_samples', 'bg_bin'], axis = 1)
    
    print('No. of NA values in the following columns: ')
    display(pd.DataFrame(temp_NA_Count.isna().sum(), columns=['NA']))

    # Calculating the number of background features and features present (sum(bg_bin) equals number of features to be removed)
    print(f"No. of Background or noise features: {len(samples)-sum(is_real_feature)}")
    print(f"No. of features after excluding noise: {sum(is_real_feature)}")

    blank_removal = samples[is_real_feature.values]
    # save to file
    blank_removal.to_csv(os.path.join(result_dir, "Blanks_Removed.csv"))

In [ ]:
print('Dimension: ',blank_removal.shape)
display(blank_removal.head())

# Step 2: Imputation

<p style='text-align: justify;'> For several reasons, real world datasets might have some missing values in it, in the form of NA, NANs or 0s. Eventhough the gapfilling step of MZmine fills the missing values, we still end up with some missing values or 0s in our feature table. This could be problematic for statistical analysis. </p> 
<p style='text-align: justify;'> In order to have a better dataset, we cannot simply discard those rows or columns with missing values as we will lose a chunk of our valuable data. Instead we can try imputing those missing values. Imputation involves replacing the missing values in the data with a meaningful, reasonable guess. There are several methods, such as: </p> 
  
1) Mean imputation (replacing the missing values in a column with the mean or average of the column)  
2) Replacing it with the most frequent value  
3) Several other machine learning imputation methods such as k-nearest neighbors algorithm(k-NN), Hidden Markov Model(HMM)

Here, we use ft and see the frquency distribution of its features with a plot. It shows where the features are present in higher number.

In [ ]:
bins, bins_label, a = [-1, 0, 1, 10], ['-1','0', "1", "10"], 2

while a<=10:
    bins_label.append(np.format_float_scientific(10**a))
    bins.append(10**a)
    a+=1

freq_table = pd.DataFrame(bins_label)
frequency = pd.DataFrame(np.array(np.unique(np.digitize(blank_removal.to_numpy(), bins, right=True), return_counts=True)).T).set_index(0)
freq_table = pd.concat([freq_table,frequency], axis=1).fillna(0).drop(0)
freq_table.columns = ['intensity', 'Frequency']
freq_table['Log(Frequency)'] = np.log(freq_table['Frequency']+1)

# get the lowest intensity (that is not zero) as a cutoff LOD value
cutoff_LOD = round(blank_removal.replace(0, np.nan).min(numeric_only=True).min())

fig = px.bar(freq_table, x="intensity", y="Log(Frequency)", template="plotly_white",  width=600, height=400)

fig.update_traces(marker_color="#696880")
fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                  title={"text":"FEATURE INTENSITY - FREQUENCY PLOT", 'x':0.5, "font_color":"#3E3D53"})
fig.write_image(os.path.join(result_dir, "frequency_plot.svg"))
fig.show()

A random number between this minimum value and zero will be used for imputation.

In [ ]:
imputed = blank_removal.copy()
if(input("Do you want to perform Imputation? - Y/N: ").upper()=="Y"):
    #imputed.replace(0, np.random.randint(0, cutoff_LOD), inplace=True)
    imputed = imputed.apply(lambda x: [np.random.randint(0, cutoff_LOD) if v == 0 else v for v in x])
    print('Dimension: ',imputed.shape)
    display(imputed)
    # save to file
    imputed.to_csv(os.path.join(result_dir, f"Imputed_QuantTable.csv"))

Too many missing values is problematic for statistical analyses. Here we calculate the proportion of missing values (coded as the value of the cutoff_LOD) and display the proportions in a histogram

TODO move plot up before imputation

In [ ]:
# check the number of missing values per feature in a histogram
n_zeros = imputed.T.apply(lambda x: sum(x<=cutoff_LOD))

fig = px.histogram(n_zeros, template="plotly_white",  
                   width=600, height=400)

fig.update_traces(marker_color="#696880")
fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                  title={"text":"MISSING VALUES PER FEATURE", 'x':0.5, "font_color":"#3E3D53"},
                  xaxis_title="number of missing values", yaxis_title="count", showlegend=False)
fig.write_image(os.path.join(result_dir, "number_of_missing_values_per_feature.svg"))
fig.show()

# Step 3 Normalization
The following code performs sample-centric (column-wise) normalisation:

In [ ]:
normalized = imputed.copy()
if(input("Do you want to perform Normalization? - Y/N: ").upper()=="Y"):
    # Dividing each element of a particular column with its column sum
    normalized = normalized.apply(lambda x: x/np.sum(x), axis=0)
    
    # save to file
    normalized.to_csv(os.path.join(result_dir, "Normalised_Quant_table.csv"))
    
    print('Dimension: ', normalized.shape)
    display(normalized.head())

# Step 4: Transposing

In [ ]:
# transposing the imputed table before scaling
transposed = imputed.T
print(f'Imputed feature table rows/columns: {transposed.shape}')
display(transposed.head(3))
# put the rows in the feature table and metadata in the same order
transposed.sort_index(inplace=True)
md_samples.sort_index(inplace=True)
try:
    print(md_samples.index == transposed.index) # should be all True
except:
    print("WARNING: Sample names in feature and metadata table are NOT the same!")
transposed.to_csv(os.path.join(result_dir, "Imputed_QuantTable_transposed.csv"))

# Step 5: Scaling
For statistics normalization should happen across the complete dataframe via scaling and centering. 

In [ ]:
print(f"Proportion of the dataset that consists of missing values (coded as {cutoff_LOD}): {((transposed<=cutoff_LOD).to_numpy()).mean()}")
print(f"\nMetabolites with measurements in at least 50 % of the samples: {(((n_zeros/transposed.shape[0])<=0.5).to_numpy()).mean()}")

# Deselect metabolites with more than 50 % missing values. This helps to get rid of features that are present in too few samples to conduct proper statistical tests
data_filtered = transposed[transposed.columns[(n_zeros/transposed.shape[0])<0.5]]

# scale filtered data
scaled = pd.DataFrame(StandardScaler().fit_transform(data_filtered), index=data_filtered.index, columns=data_filtered.columns)
scaled.to_csv(os.path.join(result_dir, "Imputed_Scaled_QuantTable.csv"))

# Merge feature table and metadata to one dataframe:
# "how=inner" performs an inner join (only the filenames that appear in md_samples and data are kept)
data = pd.merge(md_samples, scaled, left_index=True, right_index=True, how="inner")
display(data.head())

# Univariate:

**Run ANOVA** <br>

We now use the function anova from the pingouin library to run the ANOVA. Since one ANOVA is being run for each metabolite feature, we run the analyses in a loop and save the output for each feature in a list called anova_out.<br>

The vector a indicates which columns in the dataset are features (i.e. from column 5 to the last column of the data frame). <br>

We can run a for loop to pass each feature column into the first argument of the aov function, while the second argument, time point, is constant.

In [ ]:
# select an attribute to perform ANOVA
anova_attribute = 'ATTRIBUTE_Sample_Area'

def gen_anova_data(df, columns, groups_col):
    for col in columns:
        result = pg.anova(data=df, dv=col, between=groups_col, detailed=True).set_index('Source')
        p = result.loc[groups_col, 'p-unc']
        f = result.loc[groups_col, 'F']
        yield col, p, f

dtypes = [('metabolite', 'U100'), ('p', 'f'), ('F', 'f')]
anova = pd.DataFrame(np.fromiter(gen_anova_data(data, scaled.columns, anova_attribute), dtype=dtypes))
anova

The following is of interest:
*   Feature ID (column 'metabolite')
*   p-value for ANOVA
*   p-value after taking multiple tests into consideration
*   F-value

In [ ]:
# add Bonferroni corrected p-values for multiple testing correction
if 'p_bonferroni' not in anova.columns:
    anova.insert(2, 'p_bonferroni', pg.multicomp(anova['p'], method='bonf')[1])
# add significance
if 'significant' not in anova.columns:
    anova.insert(3, 'significant', anova['p_bonferroni'] < 0.05)
# sort by p-value
anova.sort_values('p', inplace=True)
# save ANOVA table
anova.to_csv(os.path.join(result_dir, 'ANOVA_results.csv'))

**Plot ANOVA results**

We will use plotly to visualize results from the ANOVA, with log(F-values) on the x-axis and -log(p) on the y-axis. Features are colored after statistical significance after multiple test correction. Since there are large differences in the F- and p-values, it is easier to plot their log.

We can also display the names of some of the top features in the plot. This easily gets very cluttered if we decide to display too many names, so starting at the top 5 could be a good idea.

In [ ]:
# first plot insignificant features
fig = px.scatter(x=anova[anova['significant'] == False]['F'].apply(np.log),
                y=anova[anova['significant'] == False]['p'].apply(lambda x: -np.log(x)),
                template='plotly_white', width=600, height=600)
fig.update_traces(marker_color="#696880")

# plot significant features
fig.add_scatter(x=anova[anova['significant']]['F'].apply(np.log),
                y=anova[anova['significant']]['p'].apply(lambda x: -np.log(x)),
                mode='markers+text',
                text=anova['metabolite'].iloc[:4],
                textposition='top left', textfont=dict(color='#ef553b', size=7), name='significant')

fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                  title={"text":"ANOVA - FEATURE SIGNIFICANCE", 'x':0.5, "font_color":"#3E3D53"},
                  xaxis_title="log(F)", yaxis_title="-log(p)", showlegend=False)

# save fig as pdf
fig.write_image(os.path.join(result_dir, "plot_ANOVA.pdf"), scale=3)

fig.show()

In [ ]:
# boxplots with top 4 metabolites from ANOVA
for metabolite in anova.sort_values('p_bonferroni').iloc[:4, 0]:
    fig = px.box(data, x=anova_attribute, y=metabolite, color=anova_attribute)
    fig.update_layout(showlegend=False, title=metabolite, xaxis_title="", yaxis_title="intensity", template="plotly_white", width=500)
    display(fig)

**Tukey's post hoc test:**

Features with both positive and negative time trends exist, but differences appear to the be largest between timepoint 0 and 45. Tukey's post hoc test can be used to calculate which features show statistically significant differences between these two timepoints.

TODO: modify to compare every location with each other

In [ ]:

# def gen_pairwise_tukey(df, time_points, metabolites):
#     """ Yield results for pairwise Tukey test for all metabolites between start and end time points."""
#     for metabolite in metabolites:
#         df_for_tukey = df.iloc[np.where(data['ATTRIBUTE_Sample_Area'].isin([time_points[0], time_points[-1]]))][[metabolite, 'ATTRIBUTE_Sample_Area']]
#         tukey = pg.pairwise_tukey(df_for_tukey, dv=metabolite, between='ATTRIBUTE_Sample_Area')
#         yield metabolite, int(metabolite.split('_')[0]), tukey['diff'], tukey['p-tukey']

# dtypes = [('stats_metabolite', 'U100'), ('stats_ID', 'i'), ('stats_diff', 'f'), ('stats_p', 'f')]
# tukey = pd.DataFrame(np.fromiter(gen_pairwise_tukey(data, time_points, anova[anova['significant']]['metabolite']), dtype=dtypes))
# # add Bonferroni corrected p-values
# tukey.insert(4, 'stats_p_bonferroni', pg.multicomp(tukey['stats_p'], method='bonf')[1])
# # add significance
# tukey.insert(5, 'stats_significant', tukey['stats_p_bonferroni'] < 0.05)
# # sort by p-value
# tukey.sort_values('stats_p', inplace=True)

# # write output to csv file
# tukey.to_csv(os.path.join(result_dir, 'TukeyHSD_output.csv'))

# display(tukey)

Create a volcano plot that displays -log(p) on the y-axis and group-difference on the x-axis. Again, display names of top findings in the plot

Create a volcano plot that displays -log(p) on the y-axis and group-difference on the x-axis. Again, display names of top findings in the plot

In [ ]:
# # create figure
# fig = px.scatter(template='plotly_white', width=600, height=600)

# # plot insignificant values
# fig.add_trace(go.Scatter(x=tukey[tukey['stats_significant'] == False]['stats_diff'],
#                          y=tukey[tukey['stats_significant'] == False]['stats_p'].apply(lambda x: -np.log(x)),
#                          mode='markers', marker_color='#696880', name='insignificant'))

# # plot significant values
# fig.add_trace(go.Scatter(x=tukey[tukey['stats_significant']]['stats_diff'],
#                          y=tukey[tukey['stats_significant']]['stats_p'].apply(lambda x: -np.log(x)),
#                          mode='markers+text', text=anova['metabolite'].iloc[:4], textposition='top left', 
#                          textfont=dict(color='#ef553b', size=8), marker_color='#ef553b', name='significant'))

# fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
#                   title={"text":"TUKEY - FEATURE DIFFERENCE", 'x':0.5, "font_color":"#3E3D53"},
#                   xaxis_title="stats_diff", yaxis_title="-log(p)")

# # save image as pdf
# fig.write_image(os.path.join(result_dir, "TukeyHSD_group0vsgroup45.pdf"), scale=3)

# fig.show()

As a sanity check we can check a few of the top metabolites by plotting them in a boxplot. Just change the input argument for y to match a name in the result list above.

In [ ]:
# def metabolite_boxplot(metabolite):
#     p_value = anova.set_index('metabolite')._get_value(metabolite, "p")
#     df = data[['ATTRIBUTE_Sample_Area', metabolite]]
#     title = str(metabolite)+"<br>p-value:"+str(p_value)
#     fig = px.box(df, x='ATTRIBUTE_Sample_Area', y=metabolite, template='plotly_white',
#                  width=800, height=600, points='all', color='ATTRIBUTE_Sample_Area')

#     fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
#                       title={"text":title, 'x':0.5, "font_color":"#3E3D53"},
#                       xaxis_title="time point", yaxis_title="intensity scales and centered")
#     fig.show()
#     return None

# # Add dropdown to create boxplot of metabolite selected (just for significant=True)
# interact(metabolite_boxplot, metabolite=sorted(list(anova["metabolite"][anova["significant"]==True])))

# PCoA PermANOVA:

Principal coordinates analysis (PCoA)

Principal coordinates analysis (PCoA) is a metric multidimensional scaling (MDS) method that attempts to represent sample dissimilarities in a low-dimensional space. It converts a distance matrix consisting of pair-wise distances (dissimilarities) across samples into a 2- or 3-D graph (Gower, 2005). Different distance metrics can be used to calculate dissimilarities among samples (e.g. Euclidean, Canberra, Minkowski). Performing a principal coordinates analysis using the Euclidean distance metric is the same as performing a principal components analysis (PCA). The selection of the most appropriate metric depends on the nature of your data and assumptions made by the metric.

Within the metabolomics field the Euclidean, Bray-Curtis, Jaccard or Canberra distances are most commonly used. The Jaccard distance is an unweighted metric (presence/absence) whereas Euclidean, Bray-Curtis and Canberra distances take into account relative abundances (weighted). Some metrics may be better suited for very sparse data (with many zeroes) than others. For example, the Euclidean distance metric is not recommended to be used for highly sparse data.

This video tutorial by StatQuest summarizes nicely the basic principles of PCoA: https://www.youtube.com/watch?v=GEn-_dAyYME

In [ ]:
#calculating Principal components
n = 10
pca = PCA(n_components=n)
pca_df = pd.DataFrame(data = pca.fit_transform(scaled), columns = [f'PC{x}' for x in range(1, n+1)])
pca_df.index = md_samples.index
pca_df

In [ ]:
# To get a scree plot showing the variance of each PC in percentage:
percent_variance = np.round(pca.explained_variance_ratio_* 100, decimals =2)

fig_bar = px.bar(x=pca_df.columns, y=percent_variance, template="plotly_white",  width=500, height=400)
fig_bar.update_traces(marker_color="#696880", width=0.5)
fig_bar.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                    title={"text":"PCA - VARIANCE", 'x':0.5, "font_color":"#3E3D53"},
                    xaxis_title="principal component", yaxis_title="variance (%)")
fig_bar.show()

TODO make the attibute colors work

In [ ]:

def pca_scatter_plot(attribute):
    title = f'PRINCIPLE COMPONENT ANALYSIS'

    df = pd.merge(pca_df[['PC1', 'PC2']], md_samples[attribute].apply(str), left_index=True, right_index=True)

    fig = px.scatter(df, x='PC1', y='PC2', template='plotly_white', width=600, height=400, color=attribute)

    fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                      title={"text":title, 'x':0.2, "font_color":"#3E3D53"},
                      xaxis_title=f'PC1 {round(pca.explained_variance_ratio_[0]*100, 1)}%',
                      yaxis_title=f'PC2 {round(pca.explained_variance_ratio_[1]*100, 1)}%')
    display(fig)

interact(pca_scatter_plot, attribute=sorted(md_samples.columns))

TODO fix the interact thing

In [ ]:
def pcoa(attribute, distance_matrix):
    # Create the distance matrix from the original data
    distance_matrix = skbio.stats.distance.DistanceMatrix(distance.squareform(distance.pdist(scaled.values, distance_matrix)))
    # perform PERMANOVA test
    permanova = skbio.stats.distance.permanova(distance_matrix, md_samples[attribute])
    permanova['R2'] = 1 - 1 / (1 + permanova['test statistic'] * permanova['number of groups'] / (permanova['sample size'] - permanova['number of groups'] - 1))
    display(permanova)
    # perfom PCoA
    pcoa = skbio.stats.ordination.pcoa(distance_matrix)
    df = pcoa.samples[['PC1', 'PC2']]
    df = df.set_index(md_samples.index)
    df = pd.merge(df[['PC1', 'PC2']], md_samples[attribute].apply(str), left_index=True, right_index=True)
    
    title = f'PRINCIPLE COORDINATE ANALYSIS'
    fig = px.scatter(df, x='PC1', y='PC2', template='plotly_white', width=600, height=400, color=attribute)

    fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                      title={"text":title, 'x':0.18, "font_color":"#3E3D53"},
                      xaxis_title=f'PC1 {round(pcoa.proportion_explained[0]*100, 1)}%',
                      yaxis_title=f'PC2 {round(pcoa.proportion_explained[1]*100, 1)}%')
    fig.show()
    
    # To get a scree plot showing the variance of each PC in percentage:
    percent_variance = np.round(pcoa.proportion_explained* 100, decimals =2)

    fig = px.bar(x=[f'PC{x}' for x in range(1, len(pcoa.proportion_explained)+1)], y=percent_variance, template="plotly_white",  width=500, height=400)
    fig.update_traces(marker_color="#696880", width=0.5)
    fig.update_layout(font={"color":"grey", "size":12, "family":"Sans"},
                      title={"text":"PCoA - VARIANCE", 'x':0.5, "font_color":"#3E3D53"},
                      xaxis_title="principal component", yaxis_title="variance (%)")#
    fig.show()

matrices = ['canberra', 'chebyshev', 'correlation', 'cosine', 'euclidean', 'hamming', 'jaccard', 'matching', 'minkowski', 'seuclidean']
interact(pcoa, attribute=sorted(md_samples.columns), distance_matrix=matrices)

# Hierarchial Clustering Algorithm:

We are now ready to perform a cluter analysis. The concept behind hierarchical clustering is to repeatedly combine the two nearest clusters into a larger cluster.

The first step consists of calculating the distance between every pair of observation points and stores it in a matrix;
1. It puts every point in its own cluster;
2. It merges the closest pairs of points according to their distances;
3. It recomputes the distance between the new cluster and the old ones and stores them in a new distance matrix;
4. It repeats steps 2 and 3 until all the clusters are merged into one single cluster. <br>

In [ ]:
fig = ff.create_dendrogram(scaled, labels=list(scaled.index))
fig.update_layout(width=700, height=500, template='plotly_white')

# save image as pdf
fig.write_image(os.path.join(result_dir, "Cluster_Dendrogram.pdf"), scale=3)
fig.show()

In [ ]:
# SORT DATA TO CREATE HEATMAP

# Compute linkage matrix from distances for hierarchical clustering
linkage_data_ft = linkage(scaled, method='complete', metric='euclidean')
linkage_data_samples = linkage(scaled.T, method='complete', metric='euclidean')

# Create a dictionary of data structures computed to render the dendrogram. 
# We will use dict['leaves']
cluster_samples = dendrogram(linkage_data_ft, no_plot=True)
cluster_ft = dendrogram(linkage_data_samples, no_plot=True)

# Create dataframe with sorted samples
ord_samp = scaled.copy()
ord_samp.reset_index(inplace=True)
ord_samp = ord_samp.reindex(cluster_samples['leaves'])
ord_samp.rename(columns={'index': 'Filename'}, inplace=True)
ord_samp.set_index('Filename', inplace=True)

# Create dataframe with sorted features
ord_ft = ord_samp.T.reset_index()
ord_ft = ord_ft.reindex(cluster_ft['leaves'])
ord_ft.rename(columns={'index': 'Feature'}, inplace=True)
ord_ft.set_index('Feature', inplace=True)

In [ ]:
#Heatmap
fig = px.imshow(ord_ft,y=list(ord_ft.index), x=list(ord_ft.columns), text_auto=True, aspect="auto",
               color_continuous_scale='PuOr_r', range_color=[-3,3])

fig.update_layout(
    autosize=False,
    width=700,
    height=800)

fig.update_yaxes(visible=False)
fig.update_xaxes(tickangle = 35)

# save image as pdf
fig.write_image(os.path.join(result_dir, "Heatmap.pdf"), scale=3)

fig.show()